## RNN LSTM Writing Sonnets

In [43]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

from google.colab import drive
drive.mount('/content/drive')
drive_dir = '/content/drive/My Drive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
text = (open(drive_dir + 'sonnets.txt').read())
text=text.lower()

Now we need to make a way to transer from characters to numbers and numbers to characters so two dictionarys are made.

In [53]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

Now we are going to take a sequence of 100 of the characters and save the label of the next character. This essentially makes our data for x and y.

In [54]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(length-seq_length):
     sequence = text[i:i + seq_length]
     label =text[i + seq_length]
     X.append([char_to_n[char] for char in sequence])
     Y.append(char_to_n[label])

Simple preprocessing for our model.

In [55]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

LSTM model.

In [56]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [69]:
model.fit(X_modified, Y_modified, epochs=10, batch_size=100)

Epoch 1/10
979/979 [==============================] - 57s 59ms/step - loss: 2.5029
Epoch 2/10
979/979 [==============================] - 59s 61ms/step - loss: 2.3486
Epoch 3/10
979/979 [==============================] - 59s 61ms/step - loss: 2.2346
Epoch 4/10
979/979 [==============================] - 59s 61ms/step - loss: 2.1409
Epoch 5/10
979/979 [==============================] - 59s 61ms/step - loss: 2.0581
Epoch 6/10
979/979 [==============================] - 59s 61ms/step - loss: 1.9867
Epoch 7/10
979/979 [==============================] - 59s 61ms/step - loss: 1.9187
Epoch 8/10
979/979 [==============================] - 59s 61ms/step - loss: 1.8555
Epoch 9/10
979/979 [==============================] - 59s 61ms/step - loss: 1.8016
Epoch 10/10
979/979 [==============================] - 59s 61ms/step - loss: 1.7416


In [70]:
model.save_weights(drive_dir + '/saved_models/text_generator_400_0.2_400_0.2_baseline.h5')

In [71]:
model.load_weights(drive_dir + '/saved_models/text_generator_400_0.2_400_0.2_baseline.h5')

Now we can print the predictions. Below a starter string is given and then the model makes predictions using softmax and the np.argmax finds the position of the highest predictions. It then appends that corresponding letter to the string from the beginning.

In [72]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [73]:
txt=""
for char in full_string:
    txt = txt+char
print(txt)

s the riper should by time decease,
 his tender heir might bear his memory:
 but thou, contracted to the world she steet,
 and there brreld with thee that think the steet,
 and there brreld with thee that think the steet,
 and there brreld with thee that think the steet,
 and there brreld with thee that think the steet,
 and there brreld with thee that think the steet,
 and there brreld with thee that think the steet,
 and there brreld with thee that think the steet,
 and there brreld with thee t
